In [8]:
# #Importing required libraries 
# from tensorflow.python.ops.ragged.ragged_tensor import RaggedTensorSpec as MyRaggedTensorSpec
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
# import keras 
import random
from sklearn.metrics import accuracy_score, f1_score
# from keras.applications.vgg16 import VGG16


from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import Callback

# import matplotlib.pyplot as plt
import matplotlib
from keras.callbacks import CSVLogger 
import tensorflow as tf
from keras.utils import get_custom_objects
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

from PIL import Image




In [2]:
image_list=os.listdir('./PageSegData1/PageImg')
image_list=[filename.split(".")[0]for filename in image_list] #use os.path.splittext() as an alternate 
random.shuffle(image_list)
file_train=image_list[0:int(0.70*len(image_list))]
file_valid=image_list[int(0.70*len(image_list)): int(0.90*len(image_list))]
file_test=image_list[int(0.90*len(image_list)):]

In [3]:
# Define val_accuracy as a custom metric

# Define val_accuracy as a custom metric
def val_accuracy(y_true, y_pred):
    y_pred = tf.round(y_pred)
    accuracy = tf.keras.metrics.binary_accuracy(y_true, y_pred)
    return tf.reduce_mean(accuracy)

val_accuracy_metric = tf.keras.metrics.MeanMetricWrapper(
    name='val_accuracy', fn=val_accuracy, dtype=tf.float32
)

# # Register the custom metric with Keras
# get_custom_objects().update({'val_accuracy': val_accuracy_metric})

# def val_loss(y_true, y_pred):
#     y_pred = tf.round(y_pred)
#     accuracy = tf.keras.metrics.binary_accuracy(y_true, y_pred)
#     return tf.reduce_mean(loss)

# val_loss_metric = tf.keras.metrics.MeanMetricWrapper(
#     name='val_loss', fn=val_loss, dtype=tf.float32
# )

# # Register the custom metric with Keras
# get_custom_objects().update({'val_accuracy': val_accuracy_metric})

In [3]:
#Helper Function Definitions 
'''
This function takes two images as input, displays them side by side, 
and titles them "Image" and "Segmented Image". 
This function is used later in the script to display images.
'''
def visualize(img,seg_img):
    """
    Visualizes image
    """
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title('Image')
    plt.subplot(1,2,2)
    plt.imshow(seg_img,cmap='gray')
    plt.title('Segmented Image')
    # plt.show()

'''
This creates a segmentation mask where each pixel in the image is assigned a label or category. 
In this case, all the pixels are initially labeled as 0, indicating that they do not belong to any category.
'''

#to convert the segmentation mask into one-hot encoded representation 
def get_segmented_img(img,n_classes): # also known as binary segmentation mask 

    seg_labels=np.zeros((512,512,1)) # creating a Numpy Array width * height*channels 
    img=cv2.resize(img,(512,512)) #resize img to height and width of 512 (same case as the gray scale thingy, do I need to preprocess ?)
    
    img=img[:,:,0] # i'm not sure if this line converts it to grey scale or not ( do I need to do the preprocessing part ? )
     # selecting only the first channel of the resized image array this effectively converts it to grayscale 
    # Image preprocessing needs to happen to convert img to gray scale
    # The preprocessing function can perform the following (color space conversion, filtering and etc). 
    # Look into the function I already have and also the research paper which talks about the best methods for preprocessing images 


    cl_list=[0,24] #NOT SURE IF I NEED THIS

    
    seg_labels[:,:,0]=(img==0).astype(int) # if the pixels in img are not 0 then set the corresponding pixel in 
    # seg_label to 1 so all non zero pixel in img have a mask of 1 and all zero pixel in img have a mask of 0
    
    # cv2.imshow('Segmentation Mask', seg_labels)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return seg_labels


def preprocess_img(img):
    img=cv2.resize(img,(320,320))
    return img

def label_isolation(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding method
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Convert the binary mask to a 3-channel image
    mask = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
    # # Convert the image to a tensor
    # image = tf.convert_to_tensor(image)

    # # Threshold the image
    # threshold = tf.constant(0.0)
    # mask = tf.greater(image, threshold)
    # mask = tf.cast(mask, tf.float32)

    return mask
# batch generator 

def batch_generator(filelist,n_classes,batch_size,augment_v):
  datagen = ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.1, #shifts the width of all the pixels of the image keza teg lay yalew to the beginning part yehedal
      height_shift_range=0.1, #distorts image a bit by shifting some pixels along a given axis
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='constant',
      cval=0.0/255.0,
    #means that the constant value used for padding or filling empty pixels 
    #in the image will be black, and that the value will be scaled to the 
    # range between 0 and 1 for compatibility with certain deep learning frameworks.
      preprocessing_function=None)

  while True:
    X=[]
    Y=[]
    for i in range(batch_size):
      fn=random.choice(filelist)
      img=cv2.imread(f'./PageSegData1/PageImg/{fn}.JPG',0)
      # img = np.zeros((img2.shape, img2.shape[1], 3), dtype=np.uint8)

      # # Set all three channels to the grayscale values
      # img[:, :, 0] = img2
      # img[:, :, 1] = img2
      # img[:, :, 2] = img2


      ret,img=cv2.threshold(img,150,255,cv2.THRESH_BINARY_INV)
      img=cv2.resize(img,(512,512))
      img=np.expand_dims(img,axis=-1)
      img=img/255

      seg=cv2.imread(f'./PageSegData1/PageSeg/{fn}_mask.png',1)
      seg=get_segmented_img(seg,2)

      if augment_v:
          seed = np.random.randint(1,100000)
          img = datagen.random_transform(img, seed=seed)
          seg = datagen.random_transform(seg, seed=seed)

      X.append(img)
      Y.append(seg)
    yield np.array(X),np.array(Y)

def plot_epochMetric(history,metric):
  train_metrics = history.history[metric]
  val_metrics = history.history['val_'+metric]
  epochs = range(1, len(train_metrics) + 1)
  plt.plot(epochs, train_metrics)
  plt.plot(epochs, val_metrics)
  plt.title('Training and validation '+ metric)
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend(["train_"+metric, 'val_'+metric])
  # plt.show()

# def plot_epochMetric(history,metric):
#     train_metrics = history.history[metric]
#     val_metrics = history.history['val_'+metric]
#     epochs = range(1, len(train_metrics) + 1)
#     fig, ax = plt.subplots()
#     ax.plot(epochs, train_metrics)
#     ax.plot(epochs, val_metrics)
#     ax.set_title('Training and validation '+ metric)
#     ax.set_xlabel("Epochs")
#     ax.set_ylabel(metric)
#     ax.legend(["train_"+metric, 'val_'+metric])
#     canvas = FigureCanvas(fig)
#     canvas.draw()
#     plot_image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
#     plot_image = plot_image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
#     # plt.close(fig)
#     return plot_image  

  
      #  train_metrics = history.history[metric]
      #  val_metrics = history.history['val_'+metric]
      #  epochs = range(1, len(train_metrics) + 1)
      #  fig,ax = plt.subplots()
      #  ax.plt.plot(epochs, train_metrics)
      #  ax.plt.plot(epochs, val_metrics)
      #  ax.plt.title('Training and validation '+ metric)
      #  ax.set_xlabel("Epochs")
      #  ax.set_ylabel(metric)
      #  ax.legend(["train_"+metric, 'val_'+metric])
      # #  plt.xlabel("Epochs")
      # #  plt.ylabel(metric)
      # #  plt.legend(["train_"+metric, 'val_'+metric])
      #  canvas = FigureCanvas(fig)
      #  canvas.draw()
      #  plot_image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
      #  plot_image = plot_image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      #  return plot_image
       
      #  size = fig.get_size_inches()*fig.dpi # size in pixels
      #  matplotlib.rc('figure', figsize=(10, 5))
      #  print(size)
       

       
      #  if filename is not None:
      #   # plt.savefig(filename, dpi=100, bbox_inches='tight', format='jpg')
      #  else:
      #     plt.show()

In [ ]:
import tensorflow as tf




path = "./PageSegData1/PageSeg/3_mask.png"

image = cv2.imread(path)
result = get_segmented_img(image,2)
result2 = label_isolation(image)

visualize(image,result)


In [11]:
# #splitting dataset

# random.shuffle(image_list)
# file_train=image_list[0:int(0.75*len(image_list))]
# file_valid=image_list[int(0.70*len(image_list)): int(0.90*len(image_list))]
# file_test=image_list[int(0.90*len(image_list)):]

# #important variables
# # metric =  ['accuracy']
# input_size = (512,512,1)


# # params = {
# #     'learning_rate': [1e-3, 1e-4, 1e-5], 
# #     'batch_size': [4,8,16],
# #     'epochs': [3, 5, 7],
# #     # 'filters': [8, 16, 32],
# #     'Steps':[100,150,200],
# #     'Weight_decay' : [0.00001, 0.0001, 0.001],
# #     # 'step' :[100,150,200],
# #     'Augment_Data': [True,False]
# # }

# val_loss_list = []
# val_acc_list = []
# # f1_list = []
# # prec_list = []
# # rec_list = []

In [4]:

#Actuall model 
# def unet(l2_weight = 0.02):
#     inputs = Input(input_size)
#     conv1 = Conv2D(1, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(inputs)
#     conv1 = Conv2D(1, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     # pool1 = Dropout(dropout_rate)(pool1)

#     conv2 = Conv2D(4, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(pool1)
#     conv2 = Conv2D(4, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     # pool2 = Dropout(dropout_rate)(pool2)

#     # conv3 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(pool2)
#     conv3 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(pool2)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     # pool3 = Dropout(dropout_rate)(pool3)

#     up4 = UpSampling2D(size=(2, 2))(pool3)
#     up4 = Conv2D(8, 2, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(up4)
#     # up4 = Concatenate()([up4, conv3])
#     up4 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(up4)
#     # up4 = Dropout(dropout_rate)(up4)

#     up5 = UpSampling2D(size=(2, 2))(up4)
#     up5 = Conv2D(4, 2, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(up5)
#     up5 = Concatenate()([up5, conv2])
#     up5 = Conv2D(4, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(up5)
#     # up5 = Dropout(dropout_rate)(up5)

#     up6 = UpSampling2D(size=(2, 2))(up5)
#     up6 = Conv2D(1, 2, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(up6)
#     up6 = Concatenate()([up6, conv1])
#     up6 = Conv2D(1, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_weight))(up6)
#     # up6 = Dropout(dropout_rate)(up6)

#     outputs = Conv2D(1, 1, activation='sigmoid')(up6)

#     model = Model(inputs=inputs, outputs=outputs)

#     return model



# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

def unet():
    input_shape = (320, 320, 1)     
    inputs = Input(input_shape)
 

    conv1 = Conv2D(1, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(1, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(4, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(4, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(8, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(8, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    up4 = UpSampling2D(size=(2, 2))(pool3)
    up4 = Conv2D(8, 2, activation='relu', padding='same')(up4)
    up4 = Concatenate()([up4, conv3])
    up4 = Conv2D(8, 3, activation='relu', padding='same')(up4)

    up5 = UpSampling2D(size=(2, 2))(up4)
    up5 = Conv2D(4, 2, activation='relu', padding='same')(up5)
    up5 = Concatenate()([up5, conv2])
    up5 = Conv2D(4, 3, activation='relu', padding='same')(up5)

    up6 = UpSampling2D(size=(2, 2))(up5)
    up6 = Conv2D(1, 2, activation='relu', padding='same')(up6)
    up6 = Concatenate()([up6, conv1])
    up6 = Conv2D(1, 3, activation='relu', padding='same')(up6)

    outputs = Conv2D(1, 1, activation='softmax')(up6)
    model = Model(inputs=[inputs], outputs=[outputs])
     
    return model


    # Compile the model
    # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model = unet()
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 320, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 320, 320, 1)  10          ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 320, 320, 1)  10          ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 160, 160, 1)  0           ['conv2d_1[0][0]']           

In [ ]:
#defining metrics and hyperparameters
batch_s = 16
test_gen = batch_generator(file_train,2,batch_s,True)
valid_gen = batch_generator(file_valid,2,batch_s,False)
n_epoch = 3
steps = 100
# steps_pre_epoch  = len(test_gen) // batch_s
# validation_steps = len(valid_gen) // batch_s
metric =  ['accuracy']
#defining the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
#training the model
mc = ModelCheckpoint('best_weight_epoch{epoch:03d}.h5', monitor=' precision, recall',
                             save_weights_only=True)
model.compile(optimizer = Adam(learning_rate= 1e-4), loss = 'binary_crossentropy', metrics= metric)
     
        # csv_logger = CSVLogger('training_history.csv',append = True)
history = model.fit(test_gen,epochs=n_epoch,steps_per_epoch=steps,validation_data=valid_gen,validation_steps=5,
          callbacks=[mc,early_stop],shuffle=1)
# model.save('my_model')


In [ ]:
img=cv2.imread(f'./PageSegData1/PageImg/60.JPG',0)
ret,img=cv2.threshold(img,150,255,cv2.THRESH_BINARY_INV)
img=cv2.resize(img,(512,512))
img= np.expand_dims(img,axis=-1)
img=np.expand_dims(img,axis=0)
pred=model.predict(img)
pred=np.squeeze(np.squeeze(pred,axis=0),axis=-1)
plt.imshow(pred,cmap='gray')

In [9]:
import gradio as gr
import tensorflow as tf
# from gradio.inputs import Group
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PIL import Image

# def process_figure(fig):
#     # create a canvas for the figure
#     canvas = FigureCanvasAgg(fig)

#     # render the figure as a numpy array
#     canvas.draw()
#     data = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
#     data = data.reshape(canvas.get_width_height()[::-1] + (3,))

#     # convert the numpy array to a PIL image
#     image = Image.fromarray(data)

#     return image

# model = tf. keras.models.load_model('./my_model')
# params = {
learning_rate = [1e-3, 1e-4, 1e-5], 
batch_size =  [4,8,16],
epochs = [5,10],
    # 'filters': [8, 16, 32],
Steps =[100,150,200],
#     'Weight_decay' : gr.inputs.CheckboxGroup[ 0.0001, 0.001],
    # 'step' :[100,150,200],
Augment_Data =  ['True','False']

# input_image = gr.inputs.Image(shape=(512, 512))

learning_rate = gr.Radio([1e-3, 1e-4, 1e-5], label = "Select a Learning Rate")
batch_size= gr.Radio([4,8,16],label= "Select a Batch Size")
epochs = gr.Radio([2,5,10,20,30], label = "Select an Epoch Value")
Steps = gr.Radio([100,150,200], label = "Select a Step Value")
Augment_Data =  gr.Radio( ["True","False"], label = "Do you want to augment the data")  

def result_display (learning_rate,batch_size,epochs,Steps,Augment_Data):    
           
        batch_s = batch_size
        n_epoch = epochs
        steps = Steps
        augment_v = Augment_Data
        # dims = (320,320)
        
        model = unet()
            # model.summary()
            # # batch_s = 16
        test_gen = batch_generator(file_train,2,batch_s,augment_v)
        valid_gen = batch_generator(file_valid,2,batch_s,augment_v)
            # print("YES 1 ")
            # n_epoch = 10
            # steps = 100
            # steps_pre_epoch  = len(test_gen) // batch_s
            # validation_steps = len(valid_gen) // batch_s
        metric =  ['accuracy']
            #defining the early stopping callback
        early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
            #training the model
        mc = ModelCheckpoint('best_weight_epoch{epoch:03d}.h5', monitor=' precision, recall',
                                        save_weights_only=True)
            # print("YES 2 ")
        model.compile(optimizer = Adam(learning_rate= learning_rate), loss = 'binary_crossentropy', metrics= metric)
            # print("YES 3 ")
                    # csv_logger = CSVLogger('training_history.csv',append = True)
        history = model.fit(test_gen,epochs=n_epoch,steps_per_epoch=steps,validation_data=valid_gen,validation_steps=5,
                    callbacks=[mc,early_stop],shuffle=1)
        
        # model.save('my_model')
        

        output_image1 = plot_epochMetric(history, 'accuracy')
        
        # output_image1f = process_figure(output_image1)
    
        # output_image2 = plot_epochMetric(history, 'loss')
        # output_image2f = process_figure(output_image2).
        # image_format = output_image1.format

        # print("THE IMAGE FORMAT IS ",image_format)
        return output_image1



iface = gr.Interface(fn=result_display, 
                     inputs=[   gr.Radio([1e-3, 1e-4, 1e-5], label = "Select a Learning Rate"),
                                gr.Radio([4,8,16],label= "Select a Batch Size"),
                                gr.Radio([2,5,10,20,30], label = "Select an Epoch Value"),
                                gr.Radio([100,150,200], label = "Select a Step Value"),
                                gr.Radio( ["True","False"], label = "Do you want to augment the data") ], 
                     outputs=gr.Image(shape=(320,320)))
iface.launch(debug=True)

In [7]:
img = plot_epochMetric(history,'accuracy')

NameError: name 'history' is not defined

In [ ]:
from PIL import Image

# Open the image
image = Image.open('image.jpg')

# Get the format of the image
image_format = image.format

# Print the format of the image
print('Image format:', image_format)